<a href="https://colab.research.google.com/github/vmarrib/fpsrate/blob/main/MVP_VANESSA_MARTINS_RIBEIRO_MACHINE_LEARNING_%26_ANALYTICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Parte 1 - Machine learning**

**Descrição**

Este projeto visa identificar as regiões com maior frequência de acidentes e as principais causas associadas, utilizando um dataset de acidentes de trânsito no Brasil, disponível no Kaggle. A análise permitirá entender quais fatores regionais e específicos de cada acidente estão mais presentes nas ocorrências, auxiliando na formulação de estratégias para a redução de acidentes e melhoria da segurança nas estradas.

**Objetivo**

Identificar padrões nas regiões e nas causas dos acidentes, de forma a destacar áreas de alto risco e causas mais frequentes de acidentes. Esses insights podem apoiar agências de trânsito e autoridades na priorização de recursos e políticas preventivas.


**O Dataset**

O dataset de acidentes de trânsito no Brasil (2017-2023) disponível no Kaggle contém informações detalhadas sobre os acidentes registrados. Aqui está uma lista das colunas incluídas no dataset e uma descrição de cada uma delas:

1. inverse_data: Data em que o acidente ocorreu, no formato YYYY-MM-DD.

2. week_day: Dia da semana em que o acidente ocorreu (ex: "Monday", "Tuesday").

3. hour: Hora do acidente no formato HH:MM:SS.

4. state: Sigla do estado brasileiro onde o acidente aconteceu (ex: "SP" para São Paulo).

5. road_id: Identificação da rodovia onde o acidente aconteceu. Pode conter siglas ou números que identificam especificamente a rodovia.

6. km: Quilômetro específico na rodovia onde o acidente ocorreu.

7. city: Nome da cidade onde o acidente aconteceu.

8. cause_of_accident: Causa registrada para o acidente (ex: "excesso de velocidade", "falta de atenção").

9. type_of_accident: Tipo de acidente registrado (ex: "colisão frontal", "capotamento", "atropelamento").

10. victims_condition: Condição das vítimas após o acidente, indicando se houve vítimas fatais, feridos ou se ninguém foi ferido (ex: "Fatal", "Feridos", "No Victims").

11. vehicles_involved: Número de veículos envolvidos no acidente.

12. latitude: Coordenada de latitude do local do acidente.

13. longitude: Coordenada de longitude do local do acidente.

14. weather_conditions: Condição climática no momento do acidente (ex: "chuva", "sol", "neblina").

15. road_conditions: Condição da pista no momento do acidente (ex: "seca", "molhada", "escorregadia").

16. traffic_conditions: Condição do tráfego na rodovia no momento do acidente (ex: "baixo", "moderado", "intenso").



**1. Download do Dataset do Kaggle**

In [ ]:

from google.colab import files
files.upload()

In [ ]:
# Mover o arquivo kaggle.json para o diretório correto e definir as permissões
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Download do dataset específico do Kaggle
!kaggle datasets download -d mlippo/car-accidents-in-brazil-2017-2023


In [ ]:
# Descompactar o arquivo zipado
!unzip car-accidents-in-brazil-2017-2023.zip


**2. Importação das Bibliotecas**

In [ ]:
# Importação das bibliotecas necessárias para manipulação de dados, visualização e machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder


**3. Carregar e Visualizar o Dataset**

In [ ]:
# Carregar o dataset em um DataFrame pandas
data = pd.read_csv('car_accidents.csv')  # Substitua pelo nome do arquivo CSV
data.head()  # Visualizar as primeiras linhas do dataset
data.info()  # Obter informações gerais sobre as colunas e tipos de dados


**4. Análise Exploratória dos Dados**

In [ ]:
# Visualizar a distribuição de acidentes por estado
plt.figure(figsize=(12, 6))
sns.countplot(x='state', data=data, order=data['state'].value_counts().index)
plt.title('Distribuição de Acidentes por Estado')
plt.xticks(rotation=90)
plt.show()

# Visualizar a distribuição de acidentes por tipo de estrada (road_id)
plt.figure(figsize=(12, 6))
sns.countplot(x='road_id', data=data, order=data['road_id'].value_counts().index)
plt.title('Distribuição de Acidentes por Tipo de Estrada')
plt.xticks(rotation=90)
plt.show()

**5. Pré-processamento dos Dados**

In [ ]:
# Preencher valores ausentes, se houver, para evitar problemas no modelo
data.fillna(method='ffill', inplace=True)

# Codificar variáveis categóricas para uso no modelo de machine learning
label_encoder = LabelEncoder()
data['state'] = label_encoder.fit_transform(data['state'])
data['road_id'] = label_encoder.fit_transform(data['road_id'])
data['type_of_accident'] = label_encoder.fit_transform(data['type_of_accident'])


**6. Transformação da Coluna Alvo (Rótulo)**

In [ ]:
# Criar uma nova coluna binária 'has_victims' que indica se houve vítimas
data['has_victims'] = data['victims_condition'].apply(lambda x: 1 if x != 'No Victims' else 0)


**7. Seleção de Features**

In [ ]:
# Seleção das features mais relevantes para o modelo
features = data[['state', 'road_id', 'type_of_accident', 'km', 'hour', 'week_day']]
target = data['has_victims']


# 8. Divisão em Conjunto de Treino e Teste

In [ ]:
# Dividir o dataset em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


**9. Construção e Treinamento do Modelo**

In [ ]:
# Inicializar o modelo Random Forest e treinar
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


**10. Avaliação do Modelo**

In [ ]:
# Fazer predições no conjunto de teste
y_pred = model.predict(X_test)

# Exibir relatório de classificação e matriz de confusão
print(classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title('Matriz de Confusão')
plt.xlabel('Predição')
plt.ylabel('Verdadeiro')
plt.show()


**11. Otimização dos Hiperparâmetros**

In [ ]:
# Importar GridSearchCV para otimizar hiperparâmetros
from sklearn.model_selection import GridSearchCV

# Definir o grid de parâmetros para o modelo
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

# Inicializar e rodar a busca em grade
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros
print("Melhores Parâmetros:", grid_search.best_params_)


**12. Análise dos Resultados e Conclusão**

In [ ]:
# Avaliar novamente o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Análise detalhada do desempenho com o modelo otimizado
print(classification_report(y_test, y_pred_best))

# Matriz de confusão do modelo otimizado
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
sns.heatmap(conf_matrix_best, annot=True, fmt='d')
plt.title('Matriz de Confusão do Modelo Otimizado')
plt.xlabel('Predição')
plt.ylabel('Verdadeiro')
plt.show()
